In [ ]:
import openai, os

In [2]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [66]:
def query(prompt, **kwargs):
    """
    wrapper for the API to save the prompt and the result
    """
    args = {
    "engine":"text-davinci-003",
    "temperature":0,
    "max_tokens":400,
    "stop":"END",
    }

    for key in kwargs:
        args[key] = kwargs[key]

    r = openai.Completion.create(prompt=prompt, **args)
    return r["choices"][0]["text"].strip()


In [67]:
query('1+1=')

'2\n\n2+2=4'

In [78]:
overviewForStoryPrompt = """ /* Generate an outline for a story

type character{{
  "name": str,
  "profession": str,
  "age": int,
}}

interface outline {{
  "setting": str, //where the section takes place
  "characters": character[], //the character names
  "conflict": str, //what conflict needs to get solved
  "plotTwist": str, //any plot twists to occur
  "resolution": str // how the story concludes
}}

const generatePlot = (theme, genre, style) : outline =>{{
    return ai.generatePlot(theme)
}}


theme = '{}'
genre = '{}'
style = '{}'
//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + generatePlot(theme, genre, style))
Out:"""

In [97]:
theme = "civil war"
genre = "movie"
style = "shakesepear"


In [98]:
rawOverview = query(overviewForStoryPrompt.format(theme, genre, style), temperature=.5)

In [99]:
rawOverview

'{\n  "setting": "During the civil war",\n  "characters": [\n    {\n      "name": "John",\n      "profession": "Soldier",\n      "age": 22\n    },\n    {\n      "name": "Jane",\n      "profession": "Nurse",\n      "age": 18\n    },\n    {\n      "name": "Jack",\n      "profession": "General",\n      "age": 55\n    }\n  ],\n  "conflict": "John and Jane must decide whether to fight for their side or flee the war",\n  "plotTwist": "Jack turns out to be John\'s father",\n  "resolution": "John and Jane decide to flee the war and start a new life together"\n}'

In [100]:
import json

In [101]:
parsedOverview = json.loads(rawOverview)

In [102]:
parsedOverview

{'setting': 'During the civil war',
 'characters': [{'name': 'John', 'profession': 'Soldier', 'age': 22},
  {'name': 'Jane', 'profession': 'Nurse', 'age': 18},
  {'name': 'Jack', 'profession': 'General', 'age': 55}],
 'conflict': 'John and Jane must decide whether to fight for their side or flee the war',
 'plotTwist': "Jack turns out to be John's father",
 'resolution': 'John and Jane decide to flee the war and start a new life together'}

In [103]:
outlineForStoryPrompt = """ /* Generate an outline for a story

type section{{
  "name": str, // the name of the section
  "description": str, // a brief overview of what happens in the section
}}

interface outline {{
  "parts": section[] //all of the sections in the story
}}

const generateOutline = (storyOverview) : outline =>{{
    return ai.generateOutline(storyOverview)
}}

overview = '{}'

//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + generateOutline(overview))
Out:"""

In [104]:
rawOutline = query(outlineForStoryPrompt.format(rawOverview), temperature=.5)

In [105]:
rawOutline

'{\n  "parts": [\n    {\n      "name": "Introduction",\n      "description": "Introduce the setting, characters, and conflict of the story."\n    },\n    {\n      "name": "Rising Action",\n      "description": "John and Julia must work together to save the Union from destruction."\n    },\n    {\n      "name": "Plot Twist",\n      "description": "Julia is revealed to be a Confederate spy, but John convinces her to switch sides."\n    },\n    {\n      "name": "Climax",\n      "description": "John and Julia lead the Union to victory, ending the Civil War."\n    },\n    {\n      "name": "Conclusion",\n      "description": "Wrap up the story and reflect on the events that took place."\n    }\n  ]\n}'

In [106]:
parsedOutline = json.loads(rawOutline)

In [107]:
parsedOutline

{'parts': [{'name': 'Introduction',
   'description': 'Introduce the setting, characters, and conflict of the story.'},
  {'name': 'Rising Action',
   'description': 'John and Julia must work together to save the Union from destruction.'},
  {'name': 'Plot Twist',
   'description': 'Julia is revealed to be a Confederate spy, but John convinces her to switch sides.'},
  {'name': 'Climax',
   'description': 'John and Julia lead the Union to victory, ending the Civil War.'},
  {'name': 'Conclusion',
   'description': 'Wrap up the story and reflect on the events that took place.'}]}

config file is an object, inputs and outputs

for each input, that's on the bottom, each output on the top

# WE STARTED TO WRITE THIS THEN REALIZED WE SKIPPING AHEAD

In [111]:
promptGenerator = """Generate typescript files to perform a function, e.g.

Purpose: generate a poem
{{
  "description": "generate a poem",
  "shortName": "generatePoem",
  "inputs": [
  {{"style": "shakespearean",
    "interpretationOfStyle": "the author type to emultae"
  }}
  ],
  "outputs": [
    {{
      "poem": "O Romeo, Romeo, wherefore art thou Romeo?\nDeny thy father and refuse thy name."
    }}
  ]
}}

Purpose: {}
"""

In [112]:
query(promptGenerator.format('generate a paragraph'))

'{\n  "description": "generate a paragraph",\n  "shortName": "generateParagraph",\n  "inputs": [\n    {\n      "style": "informative",\n      "interpretationOfStyle": "the type of information to include"\n    }\n  ],\n  "outputs": [\n    {\n      "paragraph": "The human brain is an incredibly complex organ, capable of processing vast amounts of information and making decisions in a fraction of a second. It is composed of billions of neurons, which communicate with each other through electrical and chemical signals. The brain is responsible for controlling all of the body\'s functions, from breathing and digestion to movement and emotion."\n    }\n  ]\n}'

In [123]:
fillOutlinePrompt = """ /* Write the next paragraph in a story for a given purpose

fullOverview = '{}'
fullOutline = '{}'
priorStory = '{}'

const generateNextParagraph = (instructions, currentOutlineSection) : str =>{{
    return ai.write()
}}

instructions = '{}'
currentOutlineSection = '{}'

//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + generateNextParagraph(instructions, currentOutlineSection))
Out:"""

In [117]:
story = ""
currentRawOverview = rawOverview
currentRawOutline = rawOutline

In [121]:
parsedOutline['parts'][0]['name']

'Introduction'

In [127]:
curPrompt = fillOutlinePrompt.format(rawOverview, rawOutline, story, "build the setting", parsedOutline['parts'][0]['name'])
print(curPrompt)


 /* Write the next paragraph in a story for a given purpose

fullOverview = '{
  "setting": "During the civil war",
  "characters": [
    {
      "name": "John",
      "profession": "Soldier",
      "age": 22
    },
    {
      "name": "Jane",
      "profession": "Nurse",
      "age": 18
    },
    {
      "name": "Jack",
      "profession": "General",
      "age": 55
    }
  ],
  "conflict": "John and Jane must decide whether to fight for their side or flee the war",
  "plotTwist": "Jack turns out to be John's father",
  "resolution": "John and Jane decide to flee the war and start a new life together"
}'
fullOutline = '{
  "parts": [
    {
      "name": "Introduction",
      "description": "Introduce the setting, characters, and conflict of the story."
    },
    {
      "name": "Rising Action",
      "description": "John and Julia must work together to save the Union from destruction."
    },
    {
      "name": "Plot Twist",
      "description": "Julia is revealed to be a Confedera

In [130]:
curPrompt = fillOutlinePrompt.format(rawOverview, rawOutline, story, "intro", parsedOutline['parts'][0]['name'])

query(curPrompt, temperature=.7)

"John and Jane were soldiers in the Civil War, fighting for their respective sides. Jane was a nurse, 18 years old, while John was a soldier, 22 years old. They were both caught up in the conflict and had to make a difficult decision - fight for their side or flee the war. Little did they know that a plot twist was about to unfold - Jack, the Union general, turns out to be John's father. With this knowledge, John and Jane make the difficult decision to flee the war and start a new life together."

In [168]:
def createPrompt(overview, outline, story, fnName, currentSection):
    fillOutlinePrompt = """ /* Write the next paragraph in a story for a given purpose

    //the overview of the entire story
    fullOverview = {}
    
    //the outline that the story should follow
    fullOutline = {}
    
    //the paragraphs of the story so far
    storySoFar = {}

    const {} = (instructions, currentOutlineSection) : str =>{{
        return ai.write()
    }}

    currentOutlineSection = {}

    console.log("Out: " + {}(currentOutlineSection))
    Out:""".format(overview, outline, story, fnName, currentSection, fnName)
    
    return fillOutlinePrompt

In [169]:
print(createPrompt(rawOverview, rawOutline, story, 'generateIntroduction', parsedOutline['parts'][0]['name']))



 /* Write the next paragraph in a story for a given purpose

    //the overview of the entire story
    fullOverview = {
  "setting": "During the civil war",
  "characters": [
    {
      "name": "John",
      "profession": "Soldier",
      "age": 22
    },
    {
      "name": "Jane",
      "profession": "Nurse",
      "age": 18
    },
    {
      "name": "Jack",
      "profession": "General",
      "age": 55
    }
  ],
  "conflict": "John and Jane must decide whether to fight for their side or flee the war",
  "plotTwist": "Jack turns out to be John's father",
  "resolution": "John and Jane decide to flee the war and start a new life together"
}
    
    //the outline that the story should follow
    fullOutline = {
  "parts": [
    {
      "name": "Introduction",
      "description": "Introduce the setting, characters, and conflict of the story."
    },
    {
      "name": "Rising Action",
      "description": "John and Julia must work together to save the Union from destruction."
  

In [183]:
r = query(createPrompt(rawOverview, rawOutline, story, 'generateIntroduction', parsedOutline['parts'][0]['name']))

In [184]:
print(r)

John and Jane were two brave souls in the midst of a tumultuous time. As the Civil War raged on, they were determined to do whatever it took to save their side from destruction. Little did they know, however, that their fates were intertwined with that of Jack, a 55-year-old general who had a secret connection to John. With the fate of the Union in their hands, John and Jane must decide whether to fight for their side or flee the war.


In [185]:
story = [r]

In [179]:
r = query(createPrompt(rawOverview, rawOutline, story, 'generateNextParagraph', parsedOutline['parts'][0]['name']))
story.push(r)

In [181]:
[print(x) for x in story]

It was the height of the Civil War, and the Union was in dire straits. John, a 22-year-old soldier, and Jane, an 18-year-old nurse, were determined to do whatever it took to save their side from destruction. Little did they know, however, that their fates were intertwined with that of Jack, a 55-year-old general who had a secret connection to John. With the fate of the Union in their hands, John and Jane must decide whether to fight for their side or flee the war.
John and Jane had no idea what was in store for them. As they prepared for battle, they were unaware of the danger that lurked in the shadows. Little did they know that Jack, the Union's most experienced general, was actually John's father. With the fate of the Union in their hands, John and Jane must decide whether to fight for their side or flee the war.


[None, None]

In [ ]:
class Story:
    def __init__(self, theme):
        self.theme = theme

    def doOutline(self):
        self.outline = overviewForStoryPrompt.format(theme), temperature=.5)


        
    def generateNextParagraph(self):
        r = query(self.prompt)
        self.story.append(r)
        self.currentSection = self.outline['parts'][len(self.story)]['name']
        self.prompt = createPrompt(self.overview, self.outline, self.story, self.currentSection)
        return r
    
    def __str__(self):
        return '\n'.join(self.story)